<!-- DISABLE-FRONTMATTER-SECTIONS -->

# 속도 비교

`Safetensors`는 정말 빠릅니다. [gpt2](https://huggingface.co/gpt2) 가중치를 로드하여 `PyTorch`와 비교해 보겠습니다. [GPU 벤치마크](#gpu-benchmark)를 실행하려면 머신에 GPU가 있거나 Google Colab을 사용하는 경우 `GPU 런타임`을 선택했는지 확인하세요.

시작하기 전에 필요한 모든 라이브러리가 설치되어 있는지 확인하세요.

In [ ]:
!pip install safetensors huggingface_hub torch

사용할 모든 패키지를 가져오는 것으로 시작하겠습니다.

In [ ]:
import os
import datetime
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import torch

gpt2용 safetensors 및 토치 가중치를 다운로드합니다.

In [ ]:
sf_filename = hf_hub_download("gpt2", filename="model.safetensors")
pt_filename = hf_hub_download("gpt2", filename="pytorch_model.bin")

### CPU 벤치마크

In [ ]:
start_st = datetime.datetime.now()
weights = load_file(sf_filename, device="cpu")
load_time_st = datetime.datetime.now() - start_st
print(f"Safetensors 로드 시간 {load_time_st}")

start_pt = datetime.datetime.now()
weights = torch.load(pt_filename, map_location="cpu")
load_time_pt = datetime.datetime.now() - start_pt
print(f"PyTorch 로드 시간 {load_time_pt}")

print(f"CPU에서 safetensors는 pytorch보다 {load_time_pt/load_time_st:.1f}배 빠릅니다.")

Safetensors 로드 시간 0:00:00.004015
PyTorch 로드 시간 0:00:00.307460
CPU에서 safetensors는 pytorch보다 76.6배 빠릅니다.

이러한 속도 향상은 이 라이브러리가 파일을 직접 매핑하여 불필요한 복사를 피하기 때문입니다. 실제로 [순수 pytorch](https://gist.github.com/Narsil/3edeec2669a5e94e4707aa0f901d2282)에서도 가능합니다.
현재 표시된 속도 향상은 다음에서 얻은 것입니다.
* OS: Ubuntu 18.04.6 LTS
* CPU: Intel(R) Xeon(R) CPU @ 2.00GHz

### GPU 벤치마크

In [ ]:
# 이 기능은 아직 완전히 확인되지 않았지만
# 여러 다른 환경에서 테스트되었습니다.
os.environ["SAFETENSORS_FAST_GPU"] = "1"

# 측정에서 제외된 CUDA 시작
torch.zeros((2, 2)).cuda()

start_st = datetime.datetime.now()
weights = load_file(sf_filename, device="cuda:0")
load_time_st = datetime.datetime.now() - start_st
print(f"Safetensors 로드 시간 {load_time_st}")

start_pt = datetime.datetime.now()
weights = torch.load(pt_filename, map_location="cuda:0")
load_time_pt = datetime.datetime.now() - start_pt
print(f"PyTorch 로드 시간 {load_time_pt}")

print(f"GPU에서 safetensors는 pytorch보다 {load_time_pt/load_time_st:.1f}배 빠릅니다.")

Safetensors 로드 시간 0:00:00.165206
PyTorch 로드 시간 0:00:00.353889
GPU에서 safetensors는 pytorch보다 2.1배 빠릅니다.

속도 향상은 이 라이브러리가 불필요한 CPU 할당을 건너뛸 수 있기 때문에 작동합니다. 안타깝게도 우리가 아는 한 순수 pytorch에서는 복제할 수 없습니다. 이 라이브러리는 파일을 메모리 매핑하고, pytorch로 텐서를 비어 있게 만들고, `cudaMemcpy`를 직접 호출하여 GPU로 직접 텐서를 이동하는 방식으로 작동합니다.
현재 표시된 속도 향상은 다음에서 얻은 것입니다.
* OS: Ubuntu 18.04.6 LTS.
* GPU: Tesla T4
* 드라이버 버전: 460.32.03
* CUDA 버전: 11.2